In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv


In [2]:
from collections import defaultdict

import nest_asyncio
import optuna
import pandas as pd

import os
from llama_index.core import Document

from utils.retrieve import (
    objective,
    extract_question_ngrams,
)


/home/isaiaszc/pathway/pathway-indexer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# configure
# filename = "test.md"
qa_filename = "index_single_quotes.csv"
ngram_size = 2  # use 2 instead of 3 so we don't skip 2-word header chunks
f_beta = 5  # weight recall 3 times as important as precision in f-score
n_trials = 25  # number of Optuna trials

pd.set_option("display.max_colwidth", None)


Read the question-answers


In [4]:
# qa_df = pd.read_csv(f'data/{qa_filename}', na_filter=False)
qa_df = pd.read_csv(f"../data/temporary/{qa_filename}", na_filter=False)
print(len(qa_df))
qa_df.head(3)


112


,Question,Ideal Answer,Link,Quote
0,How do I know if a student has a scholarship?,"Missionaries can see if a student has a scholarship by clicking on the student's name in Path, clicking on the Discounts/Scholarships Tab and looking under History.\n\nInformation about the Discount or Scholarship includes; Discount/Scholarship, Category, Type, Percent, Amount, Term, Status.and Process.",https://missionaries.prod.byu-pathway.psdops.com/Finding-Student-Information-in-PATH%E2%80%8B,Discounts/Scholarships Tab\nView the scholarships and/or discounts the \nstudent has received.
1,How do I know if a student is registered for an institute class for credit?,"There is not a way for Missionaries to verify that a student's current Institute class, is registered for college credit for that class.\n\nMissionaries can see if a student is currently registered for an Institute class It should appear in the Enrollment tab in the lower right hand corner. There will be a red x indicating no credit if the course has not been completed. \nEach student needs to make sure that their Institute instructor knows and has recorded that the student is listed in the course as taking it for College Credit.\n\n",https://missionaries.prod.byu-pathway.psdops.com/institute-for-college-credit,"Checking for Institute Registration for College Credit\n2. Next, click on the Enrollment tab and scroll down\n4. You will see Institute courses taken by the student as you scroll. This\r\nstudent has taken many Institute courses but has not taken them for\r\ncollege credit. The student is currently enrolled in a Spring 2024 course but\r\nshows a x NO because the course is not yet completed. Once the course is\r\ncompleted, the x No will change to a green check mark with a Yes if the\r\ncourse qualifies for credit.\n5. This student is enrolled in an institute course for Spring 2024. It is not one\r\nof the cornerstone classes. The Student should ask the instructor if this\r\ncourse will provide college credit. If so, the No will change to Yes at the\r\ncompletion of the course. The previous 2023 course taken by this student has\r\nbeen given college credit"
2,How do I know if student is member of the church?,"Missionaries can see the Church membership status by clicking on the student's name in Path. On the Student Information Page, in the Details Tab, at the right side of the page, if there are Stake and a Ward names listed, the student is a member of the church.\n\nSometimes no Stake or Ward information will be listed, but when missionaries talk to the student they may find they are members after all. In that case, missionaries can ask the student if they have another Church Account. If they can't remember, they can search for a duplicate account. Students may have to get their membership number from the ward clerk to find the real church account.\n\n",https://missionaries.prod.byu-pathway.psdops.com/Verify-a-Learner's-Church-Member-Status,"Verify a Learner’s Membership Status\n1. In the Details tab in the Student Information page, verify that a learner’s\r\nward and stake are listed if they are a member of the church.\n2. A learner may show they are a member but have no ward and stake\r\nlisted. This is a good indicator that the learner has a duplicate account.\r\no Contact your zone or district leader to investigate this further."


In [5]:
# NOTE: we shouldn't include questions in the *test* set right now,
# but people are still adding the manual quotes,
# and since we have so few questions with manual quotes so far
# we will use all of them for this demo.

# keep only rows with at least 1 manual quote
qa_df = qa_df[qa_df["Quote"].notna() & (qa_df["Quote"] != "")]
print(len(qa_df))


105


In [6]:
# generate bigrams (ngram size=2) for each manual quote
# and store them in the question_ngrams dictionary
question_ngrams = extract_question_ngrams(qa_df, ngram_size)


In [7]:
question_ngrams


{'How do I know if a student has a scholarship?': [('discounts',
   'scholarships'),
  ('scholarships', 'tab'),
  ('tab', 'view'),
  ('view', 'the'),
  ('the', 'scholarships'),
  ('scholarships', 'and'),
  ('and', 'or'),
  ('or', 'discounts'),
  ('discounts', 'the'),
  ('the', 'student'),
  ('student', 'has'),
  ('has', 'received')],
 'How do I know if a student is registered for an institute class for credit?': [('checking',
   'for'),
  ('for', 'institute'),
  ('institute', 'registration'),
  ('registration', 'for'),
  ('for', 'college'),
  ('college', 'credit'),
  ('credit', '2'),
  ('2', 'next'),
  ('next', 'click'),
  ('click', 'on'),
  ('on', 'the'),
  ('the', 'enrollment'),
  ('enrollment', 'tab'),
  ('tab', 'and'),
  ('and', 'scroll'),
  ('scroll', 'down'),
  ('down', '4'),
  ('4', 'you'),
  ('you', 'will'),
  ('will', 'see'),
  ('see', 'institute'),
  ('institute', 'courses'),
  ('courses', 'taken'),
  ('taken', 'by'),
  ('by', 'the'),
  ('the', 'student'),
  ('student', 'as')

Read the documents from the md directory


In [8]:
# Read the document names from the directories:
# ../data/markwodn
# ../data/out/md

origin_paths = ["../data/out_sep_4/from_html/", "../data/out_sep_4/from_pdf/"]

# Read the document names from the directories:
files_list = [path + item for path in origin_paths for item in os.listdir(path)]


In [9]:
files_list.sort()
files_list[0:10]


['../data/out_sep_4/from_html/-Admission-Requirements.md',
 '../data/out_sep_4/from_html/-After-PathwayConnect.md',
 '../data/out_sep_4/from_html/-Answers-to-Your-Questions-about-Ecclesiastical-Endorsement.md',
 '../data/out_sep_4/from_html/-Application-Process.md',
 '../data/out_sep_4/from_html/-Assistance-for-Students-with-Disabilities.md',
 '../data/out_sep_4/from_html/-BYU-Idaho-Course-Exceptions.md',
 '../data/out_sep_4/from_html/-BYU-Pathway-Support.md',
 '../data/out_sep_4/from_html/-BYU-Pathway-Worldwide-Website.md',
 '../data/out_sep_4/from_html/-Common-Misconceptions-about-Choosing-Certificates.md',
 '../data/out_sep_4/from_html/-Communication-Resources.md']

Save the documents into an array


In [10]:
documents = []

for i, filepath in enumerate(files_list):
    with open(filepath, "r", encoding="utf-8") as file:
        document = Document(text=file.read(), metadata={"filepath": filepath})

        # add the document to a single entry list
        documents.append(document)


In [11]:
def get_objective_fn(documents, ngram_size, question_ngrams, f_beta):
    def inner(trial):
        return objective(trial, documents, ngram_size, question_ngrams, f_beta)

    return inner


In [12]:
# ask Optuna to find the best hyperparameters

study_name = "test"  # Unique identifier of the study.
storage_name = f"sqlite:///optuna-{study_name}.db"
print(
    f"To see a dashboard, open a terminal, activate the virtual environment, and run: optuna-dashboard {storage_name}"
)
study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
    direction="maximize",
)
# study.optimize(
#     get_objective_fn(
#         documents=documents, ngram_size=ngram_size, question_ngrams=question_ngrams
#     ),
#     n_trials=n_trials,
# )
study.optimize(
    lambda trial: objective(
        trial, documents, ngram_size, question_ngrams, f_beta=f_beta
    ),
    n_trials=n_trials,
)

study.best_params


To see a dashboard, open a terminal, activate the virtual environment, and run: optuna-dashboard sqlite:///optuna-test.db


[I 2024-09-05 19:22:00,190] Using an existing study with name 'test' instead of creating a new one.


Nodes inserted: 2142


[I 2024-09-05 19:50:28,807] Trial 1 finished with value: 0.22025309614955627 and parameters: {'embed_model': 'text-embedding-3-large', 'splitter': 'semantic', 'include_prev_next_rel': True, 'buffer_size': 1, 'breakpoint_percentile_threshold': 86, 'index': 'chromadb', 'top_k': 47}. Best is trial 1 with value: 0.22025309614955627.


Nodes inserted: 993


...

Nodes inserted: 1659


[I 2024-09-06 00:20:25,346] Trial 24 finished with value: 0.275592973135385 and parameters: {'embed_model': 'text-embedding-3-large', 'splitter': 'markdown', 'include_prev_next_rel': True, 'index': 'chromadb', 'top_k': 12}. Best is trial 6 with value: 0.3130006907152221.


Nodes inserted: 1659


[I 2024-09-06 00:23:50,362] Trial 25 finished with value: 0.28474028476618973 and parameters: {'embed_model': 'text-embedding-3-large', 'splitter': 'markdown', 'include_prev_next_rel': False, 'index': 'chromadb', 'top_k': 6}. Best is trial 6 with value: 0.3130006907152221.


{'embed_model': 'text-embedding-3-large',
 'splitter': 'semantic',
 'include_prev_next_rel': False,
 'buffer_size': 1,
 'breakpoint_percentile_threshold': 81,
 'index': 'chromadb',
 'top_k': 16}